# Example: Fetch MSF API
---
This notebook indicates how to fetch the MSF api

### Imports

In [1]:
import os
os.chdir("/home/jovyan/work")

from src.utils import send_request

### Fire the request

In [2]:
result = send_request("https://api.mysportsfeeds.com/v2.0/pull/nba/2018-2019-regular/games/20181027-CHI-ATL/lineup.json")

200: OK


### Inspect the result

In [2]:
from dateutil import tz
from datetime import datetime
import json
from time import sleep

import arrow
import pandas as pd

In [3]:
# iterate games
SEASONS = [
    "2015-2016-regular",
    "2016-playoff",
    "2016-2017-regular",
    "2017-playoff",
    "2017-2018-regular"
]

def generate_game_names():
    games = []
    for season in SEASONS:
        print("Processing: {season}".format(**locals()))
        fetched_season = send_request("https://api.mysportsfeeds.com/v2.0/pull/nba/{season}/games.json".format(**locals()))
        for game in fetched_season.get("games"):
            games.append({"season": season, "game": _unmarshall_season_game(game), "id": game["schedule"]["id"]})
    return games

def _unmarshall_season_game(game):
    date = _get_datetime(game["schedule"]["homeTeam"]["abbreviation"], game["schedule"]["startTime"]).strftime("%Y%m%d")
    away = game["schedule"]["awayTeam"]["abbreviation"]
    home = game["schedule"]["homeTeam"]["abbreviation"]
    return "-".join([date, away, home])

def _get_datetime(team_abbreviaion, dt):
    if team_abbreviaion in [
        "NYK", "BRO", "BOS", "PHI", 
        "WAS", "CHA", "ATL", "ORL", 
        "MIA", "TOR", "CLE", "DET", "IND"]:
        time_zone = tz.gettz("US/Eastern")
    elif team_abbreviaion in [
        "CHI", "MIN", "DAL", "SAS", 
        "OKL", "MEM", "HOU", "MIL", "NOP"]:
        time_zone = tz.gettz("US/Central")
    elif team_abbreviaion in ["DEN", "PHX", "UTA"]:
        time_zone = tz.gettz("US/Mountain")
    elif team_abbreviaion in [
        "POR", "GSW", "LAL", "LAC", "SAC"]:
        time_zone = tz.gettz("US/Pacific")
    else:
        print("Error with timezones")
    return datetime.fromtimestamp(arrow.get(dt).timestamp, time_zone)

In [4]:
games = generate_game_names()

Processing: 2015-2016-regular
200: OK
Processing: 2016-playoff
200: OK
Processing: 2016-2017-regular
200: OK
Processing: 2017-playoff
200: OK
Processing: 2017-2018-regular
200: OK


In [5]:
def _unmarshall_lineup(lineup, game_id):
    game_lineup = {"game_id": game_id }
    team_ids = [
        team["team"]["abbreviation"]
        for team in lineup
    ]
    game_lineup.update({
        "t_{}".format(idx):t
        for (idx,t)
        in enumerate(team_ids, 1)
    })
    lu_type = "actual" if lineup[0]["actual"] else "expected"
    player_ids = [
        position["player"]["id"]
        for team in lineup
        for position in team[lu_type]["lineupPositions"]
        if position["position"].startswith("Starter") 
    ]
    game_lineup.update({
        "p_{:02d}".format(idx):p
        for (idx,p)
        in enumerate(player_ids, 1)
    })
    return game_lineup

In [6]:
def fetch_games(games):
    failed_urls = []
    for idx, g in enumerate(games, 1):
        s = "https://api.mysportsfeeds.com/v2.0/pull/nba/{season}/games/{game}/lineup.json".format(**g)

        req = send_request(s, raw=True)

        if req.status_code == 200:
            lu = json.loads(req.text).get("teamLineups")
            print("[{idx:04d}/{total}]: fetched {game}({id})".format(idx=idx, total=len(games), **g))
            PARSED_GAMES.append(_unmarshall_lineup(lu, g["id"]))
        elif req.status_code == 429:
            print("Fetched too much, sleeping for 5 minutes")
            sleep(5*60 + 30)
            req = send_request(s, raw=True)
            lu = json.loads(req.text).get("teamLineups")
            print("[{idx:04d}/{total}]: fetched {game}({id})".format(idx=idx, total=len(games), **g))
            PARSED_GAMES.append(_unmarshall_lineup(lu, g["id"]))
        else:
            print("No data for {game}({id})".format(**g))
            failed_urls.append(s)

    df = pd.DataFrame.from_records(PARSED_GAMES)
    df.to_csv("./data/generated/lineups.csv", index=False)   

In [8]:
PARSED_GAMES = []
fetch_games(games)

200: OK
[0001/3855]: fetched 20151027-DET-ATL(31176)
200: OK
[0002/3855]: fetched 20151027-CLE-CHI(31177)
200: OK
[0003/3855]: fetched 20151027-NOP-GSW(31178)
200: OK
[0004/3855]: fetched 20151028-WAS-ORL(31188)
200: OK
[0005/3855]: fetched 20151028-PHI-BOS(31179)
200: OK
[0006/3855]: fetched 20151028-CHI-BRO(31180)
200: OK
[0007/3855]: fetched 20151028-UTA-DET(31181)
200: OK
[0008/3855]: fetched 20151028-CHA-MIA(31185)
200: OK
[0009/3855]: fetched 20151028-IND-TOR(31192)
200: OK
[0010/3855]: fetched 20151028-DEN-HOU(31182)
200: OK
[0011/3855]: fetched 20151028-CLE-MEM(31184)
200: OK
[0012/3855]: fetched 20151028-NYK-MIL(31186)
200: OK
[0013/3855]: fetched 20151028-SAS-OKL(31187)
200: OK
[0014/3855]: fetched 20151028-DAL-PHX(31189)
200: OK
[0015/3855]: fetched 20151028-NOP-POR(31190)
200: OK
[0016/3855]: fetched 20151028-LAC-SAC(31191)
200: OK
[0017/3855]: fetched 20151028-MIN-LAL(31183)
200: OK
[0018/3855]: fetched 20151029-MEM-IND(31193)
200: OK
[0019/3855]: fetched 20151029-ATL-NYK(

200: OK
[0156/3855]: fetched 20151116-POR-SAS(31332)
200: OK
[0157/3855]: fetched 20151116-LAL-PHX(31331)
200: OK
[0158/3855]: fetched 20151117-MIL-WAS(31339)
200: OK
[0159/3855]: fetched 20151117-ATL-BRO(31333)
200: OK
[0160/3855]: fetched 20151117-CLE-DET(31334)
200: OK
[0161/3855]: fetched 20151117-MIN-MIA(31336)
200: OK
[0162/3855]: fetched 20151117-CHA-NYK(31338)
200: OK
[0163/3855]: fetched 20151117-DEN-NOP(31337)
200: OK
[0164/3855]: fetched 20151117-TOR-GSW(31335)
200: OK
[0165/3855]: fetched 20151118-BRO-CHA(31342)
200: OK
[0166/3855]: fetched 20151118-MIN-ORL(31345)
200: OK
[0167/3855]: fetched 20151118-IND-PHI(31346)
200: OK
[0168/3855]: fetched 20151118-DAL-BOS(31341)
200: OK
[0169/3855]: fetched 20151118-SAC-ATL(31340)
200: OK
[0170/3855]: fetched 20151118-POR-HOU(31343)
200: OK
[0171/3855]: fetched 20151118-NOP-OKL(31344)
200: OK
[0172/3855]: fetched 20151118-DEN-SAS(31348)
200: OK
[0173/3855]: fetched 20151118-TOR-UTA(31349)
200: OK
[0174/3855]: fetched 20151118-CHI-PHX(

200: OK
[0311/3855]: fetched 20151207-LAC-MIN(31483)
200: OK
[0312/3855]: fetched 20151207-BOS-NOP(31484)
200: OK
[0313/3855]: fetched 20151208-POR-CLE(31489)
200: OK
[0314/3855]: fetched 20151208-GSW-IND(31491)
200: OK
[0315/3855]: fetched 20151208-HOU-BRO(31488)
200: OK
[0316/3855]: fetched 20151208-OKL-MEM(31492)
200: OK
[0317/3855]: fetched 20151208-ORL-DEN(31490)
200: OK
[0318/3855]: fetched 20151208-UTA-SAC(31493)
200: OK
[0319/3855]: fetched 20151209-CHI-BOS(31494)
200: OK
[0320/3855]: fetched 20151209-MIA-CHA(31495)
200: OK
[0321/3855]: fetched 20151209-HOU-WAS(31503)
200: OK
[0322/3855]: fetched 20151209-MEM-DET(31497)
200: OK
[0323/3855]: fetched 20151209-SAS-TOR(31501)
200: OK
[0324/3855]: fetched 20151209-LAC-MIL(31498)
200: OK
[0325/3855]: fetched 20151209-LAL-MIN(31499)
200: OK
[0326/3855]: fetched 20151209-ORL-PHX(31500)
200: OK
[0327/3855]: fetched 20151209-NYK-UTA(31502)
200: OK
[0328/3855]: fetched 20151209-ATL-DAL(31496)
200: OK
[0329/3855]: fetched 20151210-PHI-BRO(

200: OK
[0465/3855]: fetched 20151228-MIN-SAS(31641)
200: OK
[0466/3855]: fetched 20151228-CLE-PHX(31640)
200: OK
[0467/3855]: fetched 20151228-PHI-UTA(31642)
200: OK
[0468/3855]: fetched 20151228-SAC-GSW(31636)
200: OK
[0469/3855]: fetched 20151229-MIA-MEM(32406)
200: OK
[0470/3855]: fetched 20151229-DET-NYK(31646)
200: OK
[0471/3855]: fetched 20151229-ATL-HOU(31645)
200: OK
[0472/3855]: fetched 20151229-MIL-OKL(31647)
200: OK
[0473/3855]: fetched 20151229-CLE-DEN(31644)
200: OK
[0474/3855]: fetched 20151230-LAC-CHA(31650)
200: OK
[0475/3855]: fetched 20151230-BRO-ORL(31653)
200: OK
[0476/3855]: fetched 20151230-LAL-BOS(31648)
200: OK
[0477/3855]: fetched 20151230-WAS-TOR(31657)
200: OK
[0478/3855]: fetched 20151230-IND-CHI(31649)
200: OK
[0479/3855]: fetched 20151230-UTA-MIN(31652)
200: OK
[0480/3855]: fetched 20151230-GSW-DAL(31651)
200: OK
[0481/3855]: fetched 20151230-PHX-SAS(31656)
200: OK
[0482/3855]: fetched 20151230-DEN-POR(31654)
200: OK
[0483/3855]: fetched 20151230-PHI-SAC(

200: OK
[0619/3855]: fetched 20160118-CHI-DET(31793)
200: OK
[0620/3855]: fetched 20160118-ORL-ATL(31789)
200: OK
[0621/3855]: fetched 20160118-BRO-TOR(31797)
200: OK
[0622/3855]: fetched 20160118-GSW-CLE(31791)
200: OK
[0623/3855]: fetched 20160118-BOS-DAL(31792)
200: OK
[0624/3855]: fetched 20160118-HOU-LAC(31794)
200: OK
[0625/3855]: fetched 20160119-MIL-MIA(31800)
200: OK
[0626/3855]: fetched 20160119-MIN-NOP(31801)
200: OK
[0627/3855]: fetched 20160119-OKL-DEN(31799)
200: OK
[0628/3855]: fetched 20160119-IND-PHX(31802)
200: OK
[0629/3855]: fetched 20160120-PHI-ORL(31810)
200: OK
[0630/3855]: fetched 20160120-MIA-WAS(31813)
200: OK
[0631/3855]: fetched 20160120-CLE-BRO(31803)
200: OK
[0632/3855]: fetched 20160120-UTA-NYK(31808)
200: OK
[0633/3855]: fetched 20160120-BOS-TOR(31812)
200: OK
[0634/3855]: fetched 20160120-GSW-CHI(31804)
200: OK
[0635/3855]: fetched 20160120-DET-HOU(31806)
200: OK
[0636/3855]: fetched 20160120-CHA-OKL(31809)
200: OK
[0637/3855]: fetched 20160120-MIN-DAL(

...

200: OK
[1853/3855]: fetched 20170105-CHA-DET(34478)
200: OK
[1854/3855]: fetched 20170105-ATL-NOP(34479)
200: OK
[1855/3855]: fetched 20170105-OKL-HOU(34480)
200: OK
[1856/3855]: fetched 20170105-PHX-DAL(34481)
200: OK
[1857/3855]: fetched 20170105-SAS-DEN(34483)
200: OK
[1858/3855]: fetched 20170105-LAL-POR(34482)
200: OK
[1859/3855]: fetched 20170106-MIN-WAS(34484)
200: OK
[1860/3855]: fetched 20170106-HOU-ORL(34485)
200: OK
[1861/3855]: fetched 20170106-PHI-BOS(34486)
200: OK
[1862/3855]: fetched 20170106-CLE-BRO(34487)
200: OK
[1863/3855]: fetched 20170106-NYK-MIL(34488)
200: OK
[1864/3855]: fetched 20170106-MIA-LAL(34489)
200: OK
[1865/3855]: fetched 20170106-MEM-GSW(34490)
200: OK
[1866/3855]: fetched 20170106-LAC-SAC(34491)
200: OK
[1867/3855]: fetched 20170107-NYK-IND(34492)
200: OK
[1868/3855]: fetched 20170107-NOP-BOS(34493)
200: OK
[1869/3855]: fetched 20170107-TOR-CHI(34494)
200: OK
[1870/3855]: fetched 20170107-DEN-OKL(34495)
200: OK
[1871/3855]: fetched 20170107-UTA-MIN(

200: OK
[2007/3855]: fetched 20170126-PHX-DEN(34632)
200: OK
[2008/3855]: fetched 20170126-LAL-UTA(34633)
200: OK
[2009/3855]: fetched 20170127-SAC-IND(34635)
200: OK
[2010/3855]: fetched 20170127-BRO-CLE(34636)
200: OK
[2011/3855]: fetched 20170127-ORL-BOS(34637)
200: OK
[2012/3855]: fetched 20170127-MIL-TOR(34638)
200: OK
[2013/3855]: fetched 20170127-CHA-NYK(34639)
200: OK
[2014/3855]: fetched 20170127-HOU-PHI(34634)
200: OK
[2015/3855]: fetched 20170127-MIA-CHI(34640)
200: OK
[2016/3855]: fetched 20170127-SAS-NOP(34641)
200: OK
[2017/3855]: fetched 20170127-WAS-ATL(34642)
200: OK
[2018/3855]: fetched 20170127-MEM-POR(34643)
200: OK
[2019/3855]: fetched 20170128-SAC-CHA(34644)
200: OK
[2020/3855]: fetched 20170128-DET-MIA(34645)
200: OK
[2021/3855]: fetched 20170128-BOS-MIL(34646)
200: OK
[2022/3855]: fetched 20170128-LAC-GSW(34647)
200: OK
[2023/3855]: fetched 20170128-MEM-UTA(34648)
200: OK
[2024/3855]: fetched 20170128-DEN-PHX(34649)
200: OK
[2025/3855]: fetched 20170128-BRO-MIN(

200: OK
[2162/3855]: fetched 20170215-ATL-LAC(34786)
200: OK
[2163/3855]: fetched 20170215-SAC-GSW(34787)
200: OK
[2164/3855]: fetched 20170216-WAS-IND(34788)
200: OK
[2165/3855]: fetched 20170216-BOS-CHI(34789)
200: OK
[2166/3855]: fetched 20170223-POR-ORL(34790)
200: OK
[2167/3855]: fetched 20170223-CHA-DET(34791)
200: OK
[2168/3855]: fetched 20170223-HOU-NOP(34792)
200: OK
[2169/3855]: fetched 20170223-NYK-CLE(34793)
200: OK
[2170/3855]: fetched 20170223-LAC-GSW(34794)
200: OK
[2171/3855]: fetched 20170223-DEN-SAC(34795)
200: OK
[2172/3855]: fetched 20170224-WAS-PHI(34796)
200: OK
[2173/3855]: fetched 20170224-MEM-IND(34797)
429: Too Many Requests
Fetched too much, sleeping for 5 minutes
200: OK
[2174/3855]: fetched 20170224-BOS-TOR(34798)
200: OK
[2175/3855]: fetched 20170224-PHX-CHI(34799)
200: OK
[2176/3855]: fetched 20170224-UTA-MIL(34800)
200: OK
[2177/3855]: fetched 20170224-LAL-OKL(34801)
200: OK
[2178/3855]: fetched 20170224-DAL-MIN(34802)
200: OK
[2179/3855]: fetched 201702

200: OK
[2316/3855]: fetched 20170314-IND-NYK(34941)
200: OK
[2317/3855]: fetched 20170314-POR-NOP(34942)
200: OK
[2318/3855]: fetched 20170314-PHI-GSW(34943)
200: OK
[2319/3855]: fetched 20170315-DAL-WAS(34944)
200: OK
[2320/3855]: fetched 20170315-CHA-IND(34945)
200: OK
[2321/3855]: fetched 20170315-MIN-BOS(34946)
200: OK
[2322/3855]: fetched 20170315-NOP-MIA(34947)
200: OK
[2323/3855]: fetched 20170315-UTA-DET(34948)
200: OK
[2324/3855]: fetched 20170315-MEM-CHI(34949)
200: OK
[2325/3855]: fetched 20170315-LAL-HOU(34950)
200: OK
[2326/3855]: fetched 20170315-POR-SAS(34951)
200: OK
[2327/3855]: fetched 20170315-SAC-PHX(34952)
200: OK
[2328/3855]: fetched 20170315-MIL-LAC(34953)
200: OK
[2329/3855]: fetched 20170316-UTA-CLE(34954)
200: OK
[2330/3855]: fetched 20170316-OKL-TOR(34955)
200: OK
[2331/3855]: fetched 20170316-BRO-NYK(34956)
200: OK
[2332/3855]: fetched 20170316-MEM-ATL(34957)
200: OK
[2333/3855]: fetched 20170316-LAC-DEN(34958)
200: OK
[2334/3855]: fetched 20170316-ORL-GSW(

UnboundLocalError: local variable 'response' referenced before assignment